In [ ]:
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
import numpy as np
import tensorflow as tf
from VICET import VICET
from utils import *
from scipy.spatial.transform import Rotation as R
from matplotlib import pyplot as plt
import nbconvert

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

# Run VICET on simple box enviornment

In [ ]:
#load sample data
reference_cloud = np.load("sample_point_clouds/aa.npy")
distorted_cloud = np.load("sample_point_clouds/ab.npy") #translation from a -> b in paper Fig. (1)
# distorted_cloud = np.load("sample_point_clouds/ac.npy") #translation + rotation from a -> c in paper Fig. (1) (requires some tuning of VICET parameters to get working with zero initial conditions)

# #remove ground plane
# reference_cloud = reference_cloud[reference_cloud[:,2] > -1] 
# distorted_cloud = distorted_cloud[distorted_cloud[:,2] > -1] 

#plot raw clouds
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
dpts = Points(distorted_cloud, c = "#2c7c94", alpha = 0.5).legend('Distorted Cloud')
rpts = Points(reference_cloud, c = "#CB2314", alpha = 0.5).legend('Reference Cloud')
disp.append(dpts)
disp.append(rpts)
lb = LegendBox([dpts, rpts], width=0.3, height=0.2, markers='s', bg = 'white', pos = 'top right', alpha = 0.1).font("Theemim")
plt.show(disp, lb, "raw point clouds")

In [ ]:
#seed inital guess of chi_hat
chi_init = np.array([0., 0., 0, 0, 0, 0.0,
               0, 0.0, 0, 0, 0, 0.])

chi_hat = VICET(cloud1 = reference_cloud, cloud2 = distorted_cloud, fid = 60, niter = 50, 
        draw = True, mnp = 100, RM = False, max_buffer = 3.5, chi_init = chi_init, sweep_direction = 'CW')
# sweep_direction -> VICET assumes LIDAR sensor spins Counter-Clockwise, set to CW otherwise
# fid             -> number of azimuthal voxels per 360 deg
# niter           -> number of iterations
# RM              -> Suppress voxels containing moving objects
# max_buffer      -> Max radial buffer on spherical voxels
# mnp             -> Minimum number of points required for a voxel to be considered